# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# 6 - ALS Model - Reduced Data: Grid Search + Cross-Validation

This time, as explored in the EDA NB, let's consider removing customers who we feel have too few or too many purchases to influence the model in the intended way.

Examples:
- Too few - Customers who have only bought 1 comic (series).
- Too many - Customers with > 1000 series (for example, think all eBay customers are rolled into one account number).

# Libraries

In [2]:
%matplotlib inline
%load_ext autoreload
# %autoreload 1 #would be where you need to specify the files
# %aimport comic_recs

import pandas as pd # dataframes
import os
import pickle

# Data storage
from sqlalchemy import create_engine # SQL helper
#import psycopg2 as psql #PostgreSQL DBs

# import necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import DataFrame

# Custom
import lib.data_fcns as dfc
import lib.keys  # Custom keys lib
import lib.comic_recs as cr

import time
import itertools
from functools import reduce
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# spark config
spark = SparkSession \
    .builder \
    .appName("movie recommendation") \
    .config("spark.driver.maxResultSize", "1g") \
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.master", "local[*]") \
    .getOrCreate()
# get spark context
#sc = spark.sparkContext

In [4]:
# # instantiate SparkSession object
# spark = pyspark.sql.SparkSession.builder.master("local[*]").getOrCreate()
# # spark = SparkSession.builder.master("local").getOrCreate()

## Import Data

We've previously set aside the dataset into a `json` file.

In [4]:
# We have previously created a version of the transactions table and filtered it down.
sold = spark.read.json('raw_data/als_input_filtered.json')

In [5]:
# Persist the data
sold.persist()

DataFrame[account_id: bigint, bought: bigint, comic_id: bigint]

### ALS Model

Let's start with  train/test split.

In [6]:
# Split data into training and test set
(train, test) = sold.randomSplit([.8, .2], seed=41916)

Make sure shapes make sense.

In [7]:
print(train.count(), len(train.columns))

49402 3


In [8]:
print(test.count(), len(test.columns))

12469 3


In [9]:
# Evaluate the model by computing the RMSE on the test data
eval_reg = RegressionEvaluator(metricName="rmse"
                               , labelCol="bought"
                               , predictionCol="prediction")

### Round 1

In [17]:
# hyper-param config
num_iterations = [15, 20, 30]
ranks = [5, 10, 15, 20]
reg_params = [0.01, 0.1, 0.5]
alphas = [1, 30, 40]

In [18]:
# grid search and select best model
start_time = time.time()
final_model = cr.train_ALS(train, test, eval_reg, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

15 iterations, 5 latent factors, regularization=0.01, and alpha @ 1 : validation error is 0.8419
15 iterations, 5 latent factors, regularization=0.01, and alpha @ 30 : validation error is 0.5146
15 iterations, 5 latent factors, regularization=0.01, and alpha @ 40 : validation error is 0.4816
15 iterations, 10 latent factors, regularization=0.01, and alpha @ 1 : validation error is 0.8186
15 iterations, 10 latent factors, regularization=0.01, and alpha @ 30 : validation error is 0.5055
15 iterations, 10 latent factors, regularization=0.01, and alpha @ 40 : validation error is 0.4778
15 iterations, 15 latent factors, regularization=0.01, and alpha @ 1 : validation error is 0.8083
15 iterations, 15 latent factors, regularization=0.01, and alpha @ 30 : validation error is 0.5127
15 iterations, 15 latent factors, regularization=0.01, and alpha @ 40 : validation error is 0.4907
15 iterations, 20 latent factors, regularization=0.01, and alpha @ 1 : validation error is 0.8018
15 iterations, 20

Py4JJavaError: An error occurred while calling o8924.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 75825.0 failed 1 times, most recent failure: Lost task 1.0 in stage 75825.0 (TID 94201, localhost, executor driver): java.lang.StackOverflowError
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2211)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2069)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1573)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2287)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2211)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2069)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1573)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:431)
	at scala.collection.immutable.List$SerializationProxy.readObject(List.scala:490)
	at sun.reflect.GeneratedMethodAccess

Crashed, but on inspection appears that `alpha` at 40 is best. Let's fix it at 40 and pare down some of the grid.

### Round 2

In [10]:
# hyper-param config
num_iterations = [15, 20]
ranks = [5, 10]
reg_params = [0.1, 0.5]
alphas = [40]

In [11]:
# grid search and select best model
start_time = time.time()
final_model = cr.train_ALS(train, test, eval_reg, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

15 iterations, 5 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4948
15 iterations, 10 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4968
15 iterations, 5 latent factors, regularization=0.5, and alpha @ 40 : validation RMSE is 0.4979
15 iterations, 10 latent factors, regularization=0.5, and alpha @ 40 : validation RMSE is 0.4962
20 iterations, 5 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4944
20 iterations, 10 latent factors, regularization=0.1, and alpha @ 40 : validation RMSE is 0.4951
20 iterations, 5 latent factors, regularization=0.5, and alpha @ 40 : validation RMSE is 0.4969
20 iterations, 10 latent factors, regularization=0.5, and alpha @ 40 : validation RMSE is 0.4940
Total Runtime: 103.86 seconds


- Keep around 20 iterations, and `reg_param` = 0.1

### Round 3

In [16]:
# hyper-param config
num_iterations = [20,30]
ranks = [5,10]
reg_params = [0.1]
alphas = [40]

In [17]:
# grid search and select best model
start_time = time.time()
final_model = cr.train_ALS(train, test, eval_reg, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

20 iterations, 5 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.4822
20 iterations, 10 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.4767


Py4JJavaError: An error occurred while calling o725.fit.
: org.apache.spark.SparkException: Job 238 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:932)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:930)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:930)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2128)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2041)
	at org.apache.spark.SparkContext$$anonfun$stop$6.apply$mcV$sp(SparkContext.scala:1949)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1340)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1948)
	at org.apache.spark.Spark

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 37842)
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ubuntu/anaconda3/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/ubuntu/anaconda3/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ubuntu/anaconda3/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/ubuntu/spark-2.4.3-bin-hadoop2.7/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/home/ubuntu/spark-2.4.3-bin-hadoop2.7/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/home/ubuntu/spark-2.4.3-bin-hadoop2.7/python/pyspark/accumu

### Round 4

It seems none of my assets can support going to 30 iterations. The param set using 20 iterations ran ok, but crashed on 30 I think. Or it's something else. It's probably diminishing returns to try to pursue it any further, just select best model from the parameters I *have* been able to get to run.

In [11]:
# hyper-param config
num_iterations = [25]
ranks = [5,10,15,20]
reg_params = [0.1]
alphas = [40]

In [12]:
# grid search and select best model
start_time = time.time()
final_model = cr.train_ALS(train, test, eval_reg, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

25 iterations, 5 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.4818
25 iterations, 10 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.4756
25 iterations, 15 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.4862
25 iterations, 20 latent factors, regularization=0.1, and alpha @ 40 : validation error is 0.5062
Total Runtime: 55.42 seconds


**OK**. Let's call it good. 

## Results 
Looks like the best parameters we could find are:
- `maxIter` = 25
- `rank` = 10
- `regParam` = 0.1 (default)
- `alpha` = 40

Let's do cross-validate this candidate model.

## Cross Validation

The built-in cross validator keeps breaking when I try to use it, so let's build our own function.

In [10]:
k = 5
random_seed = 41916

In [11]:
folds = cr.get_spark_k_folds(sold, k=5, random_seed=random_seed)

In [11]:
# Create ALS instance for cv with our chosen parameters
als_cv = ALS(maxIter=25,
          rank=10,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          implicitPrefs=True,
          regParam=0.1,
          alpha=40,
          coldStartStrategy='drop', # we want to drop so can get through CV
          seed=41916)

In [12]:
errors = cr.get_cv_errors(folds, als_cv, eval_reg)

In [13]:
# Make sure that # of errors = k
k == len(errors)

True

In [14]:
print(errors)

[0.4740709013236898, 0.4728406317200352, 0.4770342841276429, 0.47270879201394445, 0.4791181638842559]


In [15]:
avg_rmse = np.mean(errors)
sd_rmse = np.std(errors)
print("The average of the errors : {:.4f}".format(avg_rmse))
print("The standard deviation of the errors : {:.4f}".format(sd_rmse))


The average of the errors : 0.4752
The standard deviation of the errors : 0.0025


Looks good! Consistent with the best we've seen, and very stable.

## Candidate Model

In [241]:
# Create ALS instance and fit model
als = ALS(maxIter=25,
          rank=10,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          implicitPrefs=True,
          regParam=0.1,
          alpha=40,
          coldStartStrategy='nan', # In the final we want to find our nan's
          seed=41916)
model_use = als.fit(train)

### Get Top N recommendations for Single User

Let's make a reference list of `account_id`'s, for testing purposes.

In [12]:
n_to_test = 2

users = (sold.select(als.getUserCol())
                          .sample(False
                                  ,n_to_test/sold.count()
                                  )
        )
users.persist()
users.show(2)

+----------+
|account_id|
+----------+
|       184|
|        19|
+----------+



We developed and wrote the functionality out to a function in `comic_recs.py`

###  Testing function!

- Pass the function to a pandas dataframe. 
- Function will ask for an account_id.
- Will return top n, n defined in parameters.

In [10]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=5)
top_n_df

192
Total Runtime: 28.51 seconds


,comic_title
1,Brightest Day (DC)
2,New Mutants (Marvel)
3,Fear Itself (Marvel)
4,Batman Incorporated (DC)
5,Birds of Prey (DC)


In [11]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=5)
top_n_df

569
Total Runtime: 29.68 seconds


,comic_title
1,Thor (Marvel)
2,Superior Spider-Man (Marvel)
3,Invincible (Image)
4,Image Firsts Saga Curr Ptg (Image)
5,Superior Iron Man (Marvel)


In [12]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=10)
top_n_df

161
Total Runtime: 21.43 seconds


,comic_title
1,Aliens Dead Orbit (Dark Horse)
2,Beef (Image)
3,Vinegar Teeth (Dark Horse)
4,Leviathan (Image)
5,Shaolin Cowboy Wholl Stop th (Dark Horse)
6,Head Lopper (Image)
7,Crossed Badlands (Avatar)
8,Punisher (Marvel)
9,Godzilla Oblivion (IDW)
10,Nova (Marvel)


## Conclusions
- Seems realistic? Only three tests, but the results seem 'individualized' in the sense that there is no overlap between the sets (albeit small samples).

## Save the Model!

In [13]:
model_use.save('als_use')

## Retrieving Saved Model

In [14]:
comic_rec_model = ALSModel.load('als_use')

In [15]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=comic_rec_model, topn=10)
top_n_df

161
Total Runtime: 5.66 seconds


,comic_title
1,Aliens Dead Orbit (Dark Horse)
2,Beef (Image)
3,Vinegar Teeth (Dark Horse)
4,Leviathan (Image)
5,Shaolin Cowboy Wholl Stop th (Dark Horse)
6,Head Lopper (Image)
7,Crossed Badlands (Avatar)
8,Punisher (Marvel)
9,Godzilla Oblivion (IDW)
10,Nova (Marvel)


# New Users

Let's develop an input process that uses titles, rather than pulling the specific person's info from the modeling data.

In [14]:
comics_df = spark.read.json('support_data/comics.json')
comics_df.persist()

DataFrame[comic_id: bigint, comic_title: string]

In [100]:
sample_buys = ['Sweet Tooth', 'Paper Girls']

In [295]:
sample_buys = ['Spider-Man', 'Paper Girls']

### Get matching 'official titles'

In [325]:
def get_comic_ids_for_user(comics_df, read_comics_list):
    """
    Given spark DF of existing comics and list of comics to 'match'
    Return list of like comics from the DF
    """
    # Initialize
    similar_comics_list = []
    
    for comic in read_comics_list:
        # print(comic)
        # Search for comic in df
        matched_comics = (comics_df.filter(lower(comics_df['comic_title'])
                                 .contains(str.lower(comic)))
                                 .select('comic_id').rdd
                                 .flatMap(lambda x: x).collect()
                         )
        similar_comics_list.extend(matched_comics)
        
    return similar_comics_list


#### Testing

In [326]:
curr_comic_ids = get_comic_ids_for_user(comics_df, sample_buys)

### Add user

In [210]:
def create_acct_id(model_data):
    """
    Given model data, create new account id that is just the max existing +1
    """
    # Get max account id
    max_acct_id = model_data.agg({'account_id':'max'}).collect()[0][0]

    # New Account id
    new_acct_id = max_acct_id + 1
    
    return new_acct_id

In [211]:
def add_new_user(model_data, new_comic_ids, new_acct_id):
    """
    Given existing model data and the comic ids for new user,
    add rows for the new user to model data
    """
#     # Get max account id
#     max_acct_id = model_data.agg({'account_id':'max'}).collect()[0][0]

#     # New Account id
#     new_acct_id = max_acct_id + 1
    
    # Create spark Df of new rows
    new_rows = spark.createDataFrame([
                (new_acct_id, 1, comic_id) for comic_id in new_comic_ids])

    # Append to existing model data
    model_data_new = model_data.union(new_rows)
    
    return model_data_new

#### Testing

In [297]:
new_id = create_acct_id(sold)

In [305]:
test_sold = add_new_user(sold, curr_comic_ids, new_id)

In [306]:
test_sold.count()

61945

In [307]:
sold.count()

61871

### Train On New Data

In [178]:
# Create dictionary of candidate parameters
current_params = {'maxIter': 20
                  ,'rank': 10
                  ,'regParam': 0.1
                  ,'alpha': 40
                  ,'seed': 41916
                 }

In [331]:
def train_als(model_data, current_params):
    """
    Given training data and set of parameters
    Returns trained ALS model
    """
    # Create ALS instance for cv with our chosen parametrs
    als_train = ALS(maxIter=current_params.get('maxIter'),
              rank=current_params.get('rank'),
              userCol='account_id',
              itemCol='comic_id',
              ratingCol='bought',
              implicitPrefs=True,
              regParam=current_params.get('regParam'),
              alpha=current_params.get('alpha'),
              coldStartStrategy='nan', # we want to drop so can get through CV
              seed=41916)

    model_train = als_train.fit(model_data)
    return model_train

In [330]:
als_model = train_als(test_sold, current_params)

### Create DF of new user's unbought comics

In [332]:
def get_comics_to_rate(comics_df, training_comic_ids):
    """
    Given list of comic ids, 
    returns list of ids from master list that don't match
    """
    new_comic_ids = (comics_df.select('comic_id').distinct()
                      .filter(~col('comic_id').isin(curr_comic_ids))
                      .select('comic_id').rdd.flatMap(lambda x: x).collect()
                     )
    return new_comic_ids

In [345]:
def recommend_n_comics(top_n, new_comics_ids, account_id, als_model, comics_df):
    """
    Given a list of new comics (to the user) and requested number N
    Return list of N comics, ordered descending by recommendation score
    """

    # Create spark Df of new rows
    comics_to_predict = (spark.createDataFrame([
                        (account_id, 1, comic_id) for comic_id in new_comics_ids])
                        .select(col('_1').alias('account_id')
                        ,col('_2').alias('bought')
                        ,col('_3').alias('comic_id'))
                        )

    # Get predictions
    test_preds = als_model.transform(comics_to_predict)
    test_preds.persist()

    # Alias
    cdf = comics_df.alias('cdf')
    tp = test_preds.alias('tp')

    # Query results
    results = (tp.join(cdf, tp.comic_id==cdf.comic_id)
                .filter(~isnan(col('prediction')))
                .orderBy('prediction', ascending=False)
                .select('comic_title')
                .limit(top_n)
              ).toPandas()

    return results

In [360]:
def make_comic_recommendations(read_comics_list, top_n, comics_df, train_data 
                               ,best_params):
    """
    Given a list of comic titles and request for N
    Return list of comics recommendations as a pandas dataframe
    """
    start_time = time.time()
    
    # Get best-matching comic IDs
    train_comic_ids = get_comic_ids_for_user(comics_df, read_comics_list)
        
    # Create new account number
    new_id = create_acct_id(train_data)
    
    # Add new account to training data
    train_data_new = add_new_user(train_data, train_comic_ids, new_id)
    train_data_new.persist()
    
    # Train new ALS model
    als_model = train_als(train_data_new, best_params)
    
    # Get list of comics to rate, exclude those already matched
    new_comics_ids = get_comics_to_rate(comics_df, train_comic_ids)

    # Get pandas df of top n recommended comics!
    top_n_comics_df = recommend_n_comics(top_n, new_comics_ids, new_id
                                        ,als_model
                                        ,comics_df
                                        )
    
    print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))
    return top_n_comics_df

## BIG TEST!

In [364]:
my_list = ['Transformers', 'GI Joe', 'Y The Last Man', 'Saga', 'Avengers'
           ,'Paper Girls', 'Star Wars']

In [365]:
df = make_comic_recommendations(my_list
                               ,20
                               ,comics_df
                               ,sold
                               ,current_params)

Total Runtime: 9.53 seconds


In [366]:
df

,comic_title
0,Carnage (Marvel)
1,Deadpool Annual (Marvel)
2,Hulk (Marvel)
3,Thunderbolts (Marvel)
4,Spongebob Comics (Other)
5,Astonishing X-Men Annual (Marvel)
6,Nova (Marvel)
7,Origin Ii (Marvel)
8,Shield (Marvel)
9,Fear Itself Deadpool (Marvel)


# GRAVEYARD

In [16]:
comics_df.show(2)

+--------+--------------------+
|comic_id|         comic_title|
+--------+--------------------+
|       1|0Secret Wars (Mar...|
|       2|100 Bullets Broth...|
+--------+--------------------+
only showing top 2 rows



In [32]:
from pyspark.sql.functions import lower, upper

In [25]:
sample_comic = 'Sweet Tooth'

In [99]:
sample_comic = 'Paper Girls'

In [101]:
test = comics_df.filter(lower(comics_df['comic_title']).contains(str.lower(sample_comic))).select('comic_title')

In [102]:
test.show()

+--------------------+
|         comic_title|
+--------------------+
|Image Firsts Pape...|
| Paper Girls (Image)|
+--------------------+



In [109]:
test2 = test.withColumn('temp', col('comic_title'))

In [111]:
test2.show()

+--------------------+--------------------+
|         comic_title|                temp|
+--------------------+--------------------+
|Image Firsts Pape...|Image Firsts Pape...|
| Paper Girls (Image)| Paper Girls (Image)|
+--------------------+--------------------+



In [129]:
sold.show(2)

+----------+------+--------+
|account_id|bought|comic_id|
+----------+------+--------+
|      2247|     1|     995|
|       487|     1|    1102|
+----------+------+--------+
only showing top 2 rows



In [157]:
# Get max account id
max_acct_id = sold.agg({'account_id':'max'}).collect()[0][0]
max_acct_id

3074

In [158]:
new_acct_id = max_acct_id + 1

In [154]:
curr_comic_ids

[6105, 3235, 4701]

In [159]:
new_rows = [(new_acct_id, 1, comic_id) for comic_id in curr_comic_ids]

In [160]:
new_rows

[(3075, 1, 6105), (3075, 1, 3235), (3075, 1, 4701)]

In [161]:
sold_test = sold
sold_test.persist()


DataFrame[account_id: bigint, bought: bigint, comic_id: bigint]

In [164]:
sold_test.count()

61871

In [166]:
sold_new = sold_test.union(spark.createDataFrame(new_rows))

In [167]:
sold_new.count()

61874

In [ ]:
newRow = spark.createDataFrame([(15,'Alk','Dhl')])
df = df.union(newRow)
df.show()

In [170]:
test_sold = sold

In [171]:
test_sold_2 = add_new_user(sold, curr_comic_ids)

In [172]:
print(test_sold.count())

61871


In [173]:
print(test_sold_2.count())

61874
